In [1]:
import cartolafc
import pandas as pd
import json
import time
import requests
from functools import lru_cache

pd.set_option('display.max_columns', 50)            # permite a visualizacao de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os numeros aparecerem com duas casas decimais

# Cria uma instancia da API
api = cartolafc.Api(attempts=5)

# Constantes do 1? turno
INICIO_TURNO = 1
FIM_TURNO = 19
COLUNAS_RODADAS = [f"Rodada {r}" for r in range(INICIO_TURNO, FIM_TURNO + 1)]


2026-01-26 09:51:52,311 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Lista fixa de IDs dos participantes (lista de int)
# ids_participantes = [
#     19209079, 1488983, 186283, 16411206, 1747619,
#     32966, 44810918, 1867254, 4088673, 184499, 1273719
# ] 24856400

ids_participantes = [ 29228373, 20696550, 212042, 186283, 18642587, 1273719, 18661583, 18223508, 479510, 14124559, 19033717,
                      3851966, 44810918, 18344271, 48498051, 117598, 13707047, 3447341, 25565675, 13951133, 335716, 528730, 
                      24468241, 28741323, 24856400, 1747619, 13913874, 47544767, 51010813, 25811332, 5823700, 3708025
]

In [3]:
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json,text/plain,*/*",
    "Referer": "https://cartola.globo.com/",
}

@lru_cache(maxsize=5000)
def nome_time_por_id_api(time_id: int, timeout=15) -> str:
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}",
        f"https://api.cartolafc.globo.com/time/{time_id}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break

                data = r.json()
                if isinstance(data, dict):
                    if isinstance(data.get("time"), dict) and isinstance(data["time"].get("nome"), str):
                        return data["time"]["nome"]
                    if isinstance(data.get("nome"), str):
                        return data["nome"]
                break
            except Exception:
                time.sleep(0.5)
                continue

    return f"Time {time_id}"


In [4]:
# Base com todos os participantes
if not isinstance(ids_participantes, list) or not ids_participantes:
    raise ValueError("ids_participantes precisa ser uma lista de IDs")

ids_participantes = list(dict.fromkeys(ids_participantes))

df_base = pd.DataFrame({"time_id": ids_participantes}).drop_duplicates()
df_base["Time"] = df_base["time_id"].apply(nome_time_por_id_api)

df_base = df_base.set_index("time_id").sort_index()

# Dicionario Nome -> ID (compatibilidade com codigo legado)
ids_times_dict = {row["Time"]: row["time_id"] for _, row in df_base.reset_index().iterrows()}

# Links para o Excel
df_urls = pd.DataFrame({
    "Nome do Time": df_base["Time"].values,
    "ID do Time": df_base.index.values,
})

df_urls["Link do Time"] = df_urls["ID do Time"].apply(
    lambda x: f"https://cartola.globo.com/#!/time/{x}"
)

df_urls = df_urls[["Nome do Time", "ID do Time", "Link do Time"]]

caminho_excel = "links_times_cartola_liga_eliminacao.xlsx"
df_urls.to_excel(caminho_excel, index=False)
print(f"? Arquivo salvo com sucesso: {caminho_excel}")

display(df_base)
display(df_urls)


? Arquivo salvo com sucesso: links_times_cartola_liga_eliminacao.xlsx


,Time
time_id,
117598,A Lenda Super Vasco F.c
186283,FBC Colorado
212042,Tatols Beants F.C
335716,teves_futsal20 f.c
479510,TEAM LOPES 99
528730,Gremiomaniasm
1273719,Texas Club 2026
1747619,JV5 Tricolor Gaúcho
3447341,PUXE FC


,Nome do Time,ID do Time,Link do Time
0,A Lenda Super Vasco F.c,117598,https://cartola.globo.com/#!/time/117598
1,FBC Colorado,186283,https://cartola.globo.com/#!/time/186283
2,Tatols Beants F.C,212042,https://cartola.globo.com/#!/time/212042
3,teves_futsal20 f.c,335716,https://cartola.globo.com/#!/time/335716
4,TEAM LOPES 99,479510,https://cartola.globo.com/#!/time/479510
5,Gremiomaniasm,528730,https://cartola.globo.com/#!/time/528730
6,Texas Club 2026,1273719,https://cartola.globo.com/#!/time/1273719
7,JV5 Tricolor Gaúcho,1747619,https://cartola.globo.com/#!/time/1747619
8,PUXE FC,3447341,https://cartola.globo.com/#!/time/3447341
9,NaoVaiDescer!,3708025,https://cartola.globo.com/#!/time/3708025


In [5]:
def campeonato_comecou(ids, rodada_ref=INICIO_TURNO):
    lista_ids = list(ids.values()) if isinstance(ids, dict) else list(ids)
    for time_id in lista_ids:
        try:
            t = api.time(time_id=time_id, rodada=rodada_ref)
            v = getattr(t, "ultima_pontuacao", None)
            if v is not None:
                return True
        except Exception:
            continue
    return False


In [6]:
def safe_get_points(time_id, rodada):
    try:
        t = api.time(time_id=time_id, rodada=rodada)
        for attr in ("pontos", "pontuacao", "ultima_pontuacao"):
            v = getattr(t, attr, None)
            if v is not None:
                return float(v)
        return 0.0
    except Exception:
        return 0.0

# Rodada atual limitada ao 1? turno
try:
    rodada_atual = min(api.mercado().rodada_atual, FIM_TURNO)
except Exception:
    rodada_atual = FIM_TURNO

rodadas_ate_atual = list(range(INICIO_TURNO, rodada_atual + 1))

df_pontuacoes = df_base.copy()
for col in COLUNAS_RODADAS:
    df_pontuacoes[col] = ""

eliminados_por_rodada = {}

times_ativos = list(df_pontuacoes.index)

if not campeonato_comecou(ids_participantes, rodada_ref=INICIO_TURNO):
    print("? O campeonato ainda nao comecou. Criando tabela mock com Rodada 1 = 0.0.")
    df_pontuacoes["Rodada 1"] = 0.0
else:
    # Logica de eliminacao do 1? turno
    limite_um = INICIO_TURNO + 8  # Rodadas 1..9 eliminam 1

    for r in rodadas_ate_atual:
        print(f"? Rodada {r} ? Times participantes: {len(times_ativos)}")

        for time_id in df_pontuacoes.index:
            if time_id in times_ativos:
                pontos = safe_get_points(time_id, r)
                df_pontuacoes.at[time_id, f"Rodada {r}"] = pontos
            else:
                df_pontuacoes.at[time_id, f"Rodada {r}"] = ""

        if r < FIM_TURNO:
            n_elimina = 1 if r <= limite_um else 2
            if len(times_ativos) > 5:
                ativos_df = df_pontuacoes.loc[times_ativos].copy()
                ativos_df["pontos_rodada"] = pd.to_numeric(ativos_df[f"Rodada {r}"], errors="coerce").fillna(0.0)
                piores = ativos_df.sort_values("pontos_rodada", ascending=True).head(n_elimina)
                ids_eliminados = piores.index.tolist()
                times_ativos = [x for x in times_ativos if x not in ids_eliminados]
                eliminados_por_rodada[r] = ids_eliminados

# Exibir DataFrame final
# (mantem todos os times, mesmo sem pontuacao real)
display(df_pontuacoes)


2026-01-26 09:52:01,379 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:01,480 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:01,834 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:01,952 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:02,070 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:02,166 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:02,285 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:02,658 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:03,424 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:03,537 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:03,654 - root - ERROR - Este time ainda não foi escalado na temporada.
2026-01-26 09:52:03,772 - root -

? O campeonato ainda nao comecou. Criando tabela mock com Rodada 1 = 0.0.


,Time,Rodada 1,Rodada 2,Rodada 3,Rodada 4,Rodada 5,Rodada 6,Rodada 7,Rodada 8,Rodada 9,Rodada 10,Rodada 11,Rodada 12,Rodada 13,Rodada 14,Rodada 15,Rodada 16,Rodada 17,Rodada 18,Rodada 19
time_id,,,,,,,,,,,,,,,,,,,,
117598,A Lenda Super Vasco F.c,0.00,,,,,,,,,,,,,,,,,,
186283,FBC Colorado,0.00,,,,,,,,,,,,,,,,,,
212042,Tatols Beants F.C,0.00,,,,,,,,,,,,,,,,,,
335716,teves_futsal20 f.c,0.00,,,,,,,,,,,,,,,,,,
479510,TEAM LOPES 99,0.00,,,,,,,,,,,,,,,,,,
528730,Gremiomaniasm,0.00,,,,,,,,,,,,,,,,,,
1273719,Texas Club 2026,0.00,,,,,,,,,,,,,,,,,,
1747619,JV5 Tricolor Gaúcho,0.00,,,,,,,,,,,,,,,,,,
3447341,PUXE FC,0.00,,,,,,,,,,,,,,,,,,


In [7]:
# Exportacao (mantem formato)
excel_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx"
df_pontuacoes.to_excel(excel_path)
print(f"? Arquivo Excel salvo: {excel_path}")

# JS principal

df_dict = df_pontuacoes.fillna("").to_dict(orient="index")
js_content = f"const pontuacoesPorRodada = {json.dumps(df_dict, indent=2, ensure_ascii=False)};"

js_path = "Pontuacoes_Por_Rodada_Liga_Eliminacao.js"
with open(js_path, "w", encoding="utf-8") as f:
    f.write(js_content)

print(f"? Arquivo JS salvo: {js_path}")

# (Opcional) eliminados por rodada
try:
    elim_js = f"const eliminadosPorRodada = {json.dumps(eliminados_por_rodada, indent=2, ensure_ascii=False)};"
    with open("Eliminados_Por_Rodada_Liga_Eliminacao.js", "w", encoding="utf-8") as f:
        f.write(elim_js)
    print("? Arquivo JS salvo: Eliminados_Por_Rodada_Liga_Eliminacao.js")
except Exception:
    pass


? Arquivo Excel salvo: Pontuacoes_Por_Rodada_Liga_Eliminacao.xlsx
? Arquivo JS salvo: Pontuacoes_Por_Rodada_Liga_Eliminacao.js
? Arquivo JS salvo: Eliminados_Por_Rodada_Liga_Eliminacao.js


## Pontuacao dos Finalistas/Campeoes da Liga Eliminacao (1? turno)


In [8]:
# Soma das pontuacoes ate a rodada final do turno
rodadas_disponiveis = [col for col in df_pontuacoes.columns if col.startswith("Rodada ")]
rodadas_ate_fim = [col for col in rodadas_disponiveis if int(col.split()[-1]) <= FIM_TURNO]

# Soma apenas colunas numericas (strings vazias viram 0)
df_soma = pd.to_numeric(df_pontuacoes[rodadas_ate_fim], errors="coerce").fillna(0.0).sum(axis=1)

df_classificacao_rodada_final = pd.DataFrame({
    "ID": df_pontuacoes.index,
    "Time": df_pontuacoes["Time"],
    f"Total ate Rodada {FIM_TURNO}": df_soma,
}).sort_values(by=f"Total ate Rodada {FIM_TURNO}", ascending=False).reset_index(drop=True)

print(df_classificacao_rodada_final)


TypeError: arg must be a list, tuple, 1-d array, or Series